In [1]:
#Задание 1
import math
import numpy as np
import scipy.linalg
from scipy.linalg import sqrtm
from scipy.stats.distributions import chi2

def neymann(f,a,b,M): 
    while 1==1:
        u1=np.random.random() # u1 и u2 две независимые равномерно на [0,1] распределенные случайные величины
        u2=np.random.random() 
        x1=a+(b-a)*u1
        x2=M*u2
        if x2<=f(x1):
            return x1
#N - размер выборки
#n - количество интервалов
N=1000
n=int(3.222*math.log10(N)) # количество интервалов определяется по формуле Стреджесса
x=np.zeros(N) 
for i in range(N):
    x[i]=neymann(lambda x: (2+math.cos(x))/4/math.pi, 0,2*math.pi,0.75/math.pi)
a=[2*math.pi*i/n for i in range(n+1)]
F=lambda x: x/2/math.pi+math.sin(x)/4/math.pi
emp_chi2=0
for i in range(n):
    theor_fr=(F(a[i+1])-F(a[i]))*N
    emp_fr=len([y for y in x if (y>=a[i])and(y<a[i+1])])
    emp_chi2+=(theor_fr-emp_fr)*(theor_fr-emp_fr)/theor_fr
print("Chi2:",emp_chi2)
print("Критическое значение:",chi2.ppf(0.975, df=n-1))
if emp_chi2<chi2.ppf(0.975, df=n-1):
    print("Гипотеза подтвердилась")
else:
    print("Гипотеза опроверглась")
        

Chi2: 10.835178972509127
Критическое значение: 17.534546139484647
Гипотеза подтвердилась


In [2]:
#Задание 2
C=[[1,0,1,0.5],[0,1,1,0],[1,1,2,0],[0.5,0,0,4]]
mu=[1,2,3,4]

d=len(mu)
N=1000

x=np.zeros((N,d))
A=sqrtm(C)
for i in range(N):
    x[i,:]=np.dot(A,np.random.normal(0,1,d))+mu
    

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: ComplexWarning: Casting complex values to real discards the imaginary part
  # This is added back by InteractiveShellApp.init_path()


In [3]:
i=0
j=3
r0=C[i][j]/math.sqrt(C[i][i]*C[j][j])
r=np.corrcoef(x[:,i],x[:,j])[0][1]
print("r=",r)
print("r0=",r0)
S=0.5*abs(math.log((1+r)/(1-r)*(1-r0)/(1+r0)))*math.sqrt(N-3) 
S_crit=scipy.stats.norm.ppf(0.975) 
print("Статистическое значение:",S)
print("Критическое значение:",S_crit)
if S<S_crit:
    print("Коэффициент корреляции соответствует теоретическому значению")
else:
    print("Коэффициент корреляции отличается от теоретического значения")

r= 0.26419092666889227
r0= 0.25
Статистическое значение: 0.47980755201303976
Критическое значение: 1.959963984540054
Коэффициент корреляции соответствует теоретическому значению


In [4]:
#Задание2
def inside_triangle(X,tr):
    for [A,B,C] in [[tr[0],tr[1],tr[2]],[tr[0],tr[2],tr[1]],[tr[1],tr[2],tr[0]]]:
        b=[B[0]-A[0],B[1]-A[1]]
        c=[C[0]-A[0],C[1]-A[1]]
        x=[X[0]-A[0],X[1]-A[1]]
        bc=b[0]*c[1]-b[1]*c[0]
        bx=b[0]*x[1]-b[1]*x[0]
        if bc*bx<0:
            return False
    return True
tr1=[[-1,-1],[0,1],[1,0]]
tr2=[[0,0],[0,3],[1.5,-4]]
A=np.max([abs(tr1[i][j]) for i in range(3) for j in range(2)]+[abs(tr2[i][j]) for i in range(3) for j in range(2)])
N=10000
S=0
for i in range(N):
    u1=np.random.random()
    u2=np.random.random()
    X=[2*A*u1-A,2*A*u2-A]
    if inside_triangle(X,tr1) and inside_triangle(X,tr2):
        S+=4*A**2/N
print("Площадь пересечения:", S)

Площадь пересечения: 0.5056000000000005


In [5]:
#Задание3
def capacity(v): #скорость ветра v по закону
    if (v>=4) and (v<15):
        return math.cos(v*math.pi/11+7*math.pi/11)+1 
    if (v>=15) and (v<25):
        return 7/4+v/30+v**2/900
    return 0
M=200
betta=1.5
lamb=100
caps=np.zeros(M)
for i in range(M):
    x=lamb*(-math.log(1-np.random.random()))**(1/betta)
    caps[i]=capacity(x)
m, se = np.mean(caps), scipy.stats.sem(caps)
h = se * scipy.stats.t._ppf((1+0.95)/2., M-1)
print("Распределение Вейбулла:",[m-h,m+h])

a=2
b=20

for i in range(M):
    while 1==1:
        x=lamb*(-math.log(1-np.random.random()))**(1/betta)
        if (x>a) and (x<=b):
            break
    caps[i]=capacity(x) 
m, se = np.mean(caps), scipy.stats.sem(caps)
h = se * scipy.stats.t._ppf((1+0.95)/2., M-1)
print("Усеченное до интервала распределение Вейбулла:",[m-h,m+h])


Распределение Вейбулла: [0.16415768722402319, 0.3831698274907673]
Усеченное до интервала распределение Вейбулла: [1.4536168223732149, 1.7314402717109247]


In [6]:
#Задание 4
R=0.15
dE=200
p1=0.6
p2=0.5
l=0.1
#N - текущее количество нейтронов, M - количество экспериментов
M=10000
S=0
for k in range(M):
    neutrons=[[0,0]]
    N=1
    E=0
    while E<1000000:
        for i in range(N):
            x=neutrons[i]
            dist=l*math.log(1-np.random.random())
            ksi=np.random.normal()
            t=ksi/np.linalg.norm(ksi)
            x=x+dist*t
            neutrons[i]=x
            if np.linalg.norm(x)>R:
                neutrons[i]=[]
            else:
                if np.random.random()>p1:
                    E+=dE
                    neutrons[i]=[]
                    neutrons=neutrons+[x]+[x]
                    if np.random.random()>p2:
                        neutrons=neutrons+[x]
        neutrons=[x for x in neutrons if len(x)==2]
        N=len(neutrons)
        if N==0:
            break
    if E>=1000000:
        print("Неконтролируемая ядерная реакция/выделенная энергия > 10^6")
    S+=E/M
print("Среднее количество энергии:")
print(S)

Среднее количество энергии:
987.2599999999743


In [7]:
#Задание5
N=17
M=100000
S=0
for i in range(M):
    x=[[0,0]]
    w=1
    for step in range(N):
        neighbors=[[x[-1][0]+1,x[-1][1]],[x[-1][0]-1,x[-1][1]],[x[-1][0],x[-1][1]+1],[x[-1][0],x[-1][1]-1]]
        neighbors=[z for z in neighbors if z not in x]
        d=len(neighbors)
        if d==0:
            w=0
            break
        x.append(neighbors[np.random.randint(0,d)])
        w=w*d
    S=S+w
S/M

46407199.76496

In [8]:
#Задание 6 
import numpy as np
x0=1
y0=1
z0=1
R=1.2
S0=(x0**2+y0**2+z0**2-R**2)**0.5

N=10000 #размер выборки
I=0
for i in range(N):
    u1=np.random.random()
    u2=np.random.random()
    phi=u1*2*math.pi
    theta=np.arcsin(u2**0.5)
    x=math.cos(phi)*math.sin(theta)
    y=math.sin(phi)*math.sin(theta)
    z=math.cos(theta)
    S=x*x0+y*y0+z*z0
    if S>=S0:
        I+=math.pi/N
print("Значение выборки:")
print(I)
print("Дисперсия:")
print(I*(math.pi-I)/N)
#Стандартный метод Монте-Карло
I=0
I2=0
for i in range(N):
    u1=np.random.random()
    u2=np.random.random()
    phi=u1*2*math.pi
    theta=np.arccos(1-u2)
    x=math.cos(phi)*math.sin(theta)
    y=math.sin(phi)*math.sin(theta)
    z=math.cos(theta)
    S=x*x0+y*y0+z*z0
    if S>=S0:
        I+=2*math.pi/N*z
        I2+=2*math.pi/N*z**2
print("Стандартный метод Монте-Карло:")
print(I)    
print("Дисперсия:")
print(1/N*(2*math.pi*I2-I**2))

Значение выборки:
0.8771326688822365
Дисперсия:
0.00019862318299635677
Стандартный метод Монте-Карло:
0.8802908740222377
Дисперсия:
0.00028801931804885194
